## requirements

In [1]:
%%capture

!pip install diffusers["torch"] transformers
!pip install gradio

## processing.py

In [2]:
%%capture

import scipy
import torch
from diffusers import AudioLDMPipeline

In [8]:
%%capture

# initiate model
repo_id = "cvssp/audioldm-s-full-v2"
pipe = AudioLDMPipeline.from_pretrained(repo_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# processing
def processing(prompt:str, name_file: str):
  prompt = prompt
  audio = pipe(prompt, num_inference_steps=10, audio_length_in_s=180).audios[0]

  scipy.io.wavfile.write(f'{name_file}.mp3', rate=16000, data=audio)

  audio_file = f'{name_file}.mp3'

  return audio_file

## app.py

In [ ]:
import gradio as gr

gui = gr.Interface(
    fn=processing,
    inputs=["textbox", "textbox"],
    outputs="audio"
    )

gui.launch(debug=True)